## Demonstration of control on Myeloid model

The model we will observe is coming from Krumsiek, J., Marr, C., Schroeder, T., and Theis, F. J. (2011). Hierarchical differentiation of myeloid progenitors is encoded in the transcription factor network. PloS one, 6(8), e22649.

It describes myeloid cell differentiation into four possible types - erythrocytes, megakaryocytes, monocytes, and granulocytes.

![Myeloid_differentiation](myeloid.png)

In [19]:
import biodivine_aeon as ba 
import pandas as pd

In [20]:
ba.LOG_LEVEL = ba.LOG_NOTHING

In [21]:
# Load the model and create a Perturbation Graph

model = ba.BooleanNetwork.from_file('myeloid_witness.aeon')
pstg = ba.AsynchronousPerturbationGraph(model)

In [22]:
# Find attractors representing the four possible phenotypes

attractors = ba.Attractors.attractors(pstg)
attractors = [ a.vertices() for a in attractors ]

erythrocyte = pstg.mk_subspace_vertices({ "EKLF": True })
erythrocyte_att = [ a for a in attractors if not a.intersect(erythrocyte).is_empty() ][0]
print('erythrocyte attractor size (should be 1):', erythrocyte_att.cardinality())
print('erythrocyte attractor:', [sorted(x.to_named_dict().items()) for x in  erythrocyte_att.items()])

megakaryocyte = pstg.mk_subspace_vertices({ "Fli1": True })
megakaryocyte_att = [ a for a in attractors if not a.intersect(megakaryocyte).is_empty() ][0]
print('megakaryocyte attractor size (should be 1):', megakaryocyte_att.cardinality())
print('megakaryocyte megakaryocyte:', [sorted(x.to_named_dict().items()) for x in  megakaryocyte_att.items()])

monocyte = pstg.mk_subspace_vertices({ "cJun": True })
monocyte_att = [ a for a in attractors if not a.intersect(monocyte).is_empty() ][0]
print('monocyte attractor size (should be 1):', monocyte_att.cardinality())
print('monocyte attractor:', [sorted(x.to_named_dict().items()) for x in  monocyte_att.items()])


granulocyte = pstg.mk_subspace_vertices({ "Gfi1": True })
granulocyte_att = [ a for a in attractors if not a.intersect(granulocyte).is_empty() ][0]
print('granulocyte attractor size (should be 1):', granulocyte_att.cardinality())
print('granulocyte attractor:', [sorted(x.to_named_dict().items()) for x in  granulocyte_att.items()])

erythrocyte attractor size (should be 1): 1
erythrocyte attractor: [[('CEBPa', False), ('EKLF', True), ('EgrNab', False), ('FOG1', True), ('Fli1', False), ('GATA1', True), ('GATA2', False), ('Gfi1', False), ('PU1', False), ('SCL', True), ('cJun', False)]]
megakaryocyte attractor size (should be 1): 1
megakaryocyte megakaryocyte: [[('CEBPa', False), ('EKLF', False), ('EgrNab', False), ('FOG1', True), ('Fli1', True), ('GATA1', True), ('GATA2', False), ('Gfi1', False), ('PU1', False), ('SCL', True), ('cJun', False)]]
monocyte attractor size (should be 1): 1
monocyte attractor: [[('CEBPa', False), ('EKLF', False), ('EgrNab', True), ('FOG1', False), ('Fli1', False), ('GATA1', False), ('GATA2', False), ('Gfi1', False), ('PU1', True), ('SCL', False), ('cJun', True)]]
granulocyte attractor size (should be 1): 1
granulocyte attractor: [[('CEBPa', True), ('EKLF', False), ('EgrNab', False), ('FOG1', False), ('Fli1', False), ('GATA1', False), ('GATA2', False), ('Gfi1', True), ('PU1', True), ('SCL'

In [23]:
# Let's demonstrate now, how we can compute source-target control.

# First we need to compute a symbolic structure containing the perurbations and results
sym_results = ba.Control.attractor_one_step(pstg, erythrocyte_att, megakaryocyte_att)
sym_results

ColoredPerturbationSet(cardinality=5832, colors=1, perturbations=5832, symbolic_size=12)

In [24]:
# Now we can either look up for what colors does a specific perturbation work
sym_results.select_perturbation({ "EKLF" : True }) # A perturbation where only EKLF is perturbed to 1.
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [25]:
sym_results.select_perturbation({ "EKLF" : False }) # A perturbation where only EKLF is perturbed to 0
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [26]:
sym_results.select_perturbation({ "EKLF" : None }) # A perturbations where EKLF is unperturbed (and everything else is also unperturbed).
# Cardinality 0 -> not working

ColorSet(cardinality=0, symbolic_size=1)

In [27]:
sym_results.select_perturbation({ "EKLF" : False, "Fli1": True }) # A perturbation that we know works.

ColorSet(cardinality=1, symbolic_size=13)

In [28]:
# Or just retrieve all smallest colours with a given minimal robustness to avoid guessing
(p, r, c) = sym_results.select_by_robustness(threshold=0.99, result_limit=10)[0]
print(p.perturbed_named_dict())

{'EKLF': False, 'Fli1': True}


In [29]:
# In a similar manner, we can also compute phenotype control. Nonetheless, providing just the target state 
# (or a set of states) is sufficient in this case)

# There are three possible configurations of oscillation
# Forbidden -> attractor where model stabilizes therefore cannot oscillate through the phenotype (must stabilize fully within the phenotype)
# Allowed -> attractor where model stabilizes may oscillate
# Required -> attractor where model stabilizes MUST oscillate through the phenotype (i.e. have states both inside and outside of the phenotype space)

# In this demonstration we use forbidden type of oscillation, since myeloid model does not show oscillations 
# and this type is most similar to the source-target control.

# Even though the model is small, using ceiled version of the function is recommended, since it allows stop_early 
# functionality, computing the perturbations until 100% robustness is hit or specifying a sensible size_bounc
phen_sym_results = ba.Control.phenotype_permanent(
    graph=pstg,
    phenotype=megakaryocyte_att,
    oscillation_type="forbidden",
    size_limit=10,
    stop_when_found=True
)

In [30]:
(p, r, c) = phen_sym_results.select_by_robustness(threshold=0.99, result_limit=1)[0]
print(p.perturbed_named_dict())

{'Fli1': True, 'PU1': False}


In [31]:
# Now we can compute a full sets of results for all pairs of source-target control

In [32]:
all_atts = {
    "Erythrocyte": erythrocyte_att,
    "Megakaryocyte": megakaryocyte_att,
    "Monocyte": monocyte_att,
    "Granulocyte": granulocyte_att
}

# Generate al pair possibilities for computing the source-target control
all_att_pairs = [(s,t) for s in all_atts.keys() for t in all_atts.keys() if s != t]

In [33]:
# Pick the minimal perturbations with perfect robustness and then only return the
# values of variables that are actually perturbed (to simplify presentation).
# We are not returning the robustness here becasue it is always 1.0 anyway.
def pick_result(data: ba.ColoredPerturbationSet) -> dict[str, bool]:
    (min_model, _, _) = data.select_by_robustness(threshold=1.0, result_limit=1)[0]
    picked = data.select_by_size(size=min_model.perturbation_size(), up_to=False)
    picked = picked.select_by_robustness(threshold=1.0, result_limit=100)
    return [ model.perturbed_named_dict() for (model, _, _) in picked ]

data = {
    'source': [s for s, _ in all_att_pairs],
    'target': [t for _, t in all_att_pairs],
    'onestep': [pick_result(ba.Control.attractor_one_step(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
    'temporary': [pick_result(ba.Control.attractor_temporary(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
    'permanent': [pick_result(ba.Control.attractor_permanent(pstg, all_atts[s], all_atts[t])) for s, t in all_att_pairs],
}

df = pd.DataFrame(data)

In [34]:
df

source         target  \
0     Erythrocyte  Megakaryocyte   
1     Erythrocyte       Monocyte   
2     Erythrocyte    Granulocyte   
3   Megakaryocyte    Erythrocyte   
4   Megakaryocyte       Monocyte   
5   Megakaryocyte    Granulocyte   
6        Monocyte    Erythrocyte   
7        Monocyte  Megakaryocyte   
8        Monocyte    Granulocyte   
9     Granulocyte    Erythrocyte   
10    Granulocyte  Megakaryocyte   
11    Granulocyte       Monocyte   

                                              onestep  \
0                     [{'EKLF': False, 'Fli1': True}]   
1                                     [{'PU1': True}]   
2     [{'CEBPa': True, 'GATA1': False, 'Gfi1': True}]   
3                     [{'Fli1': False, 'EKLF': True}]   
4                                     [{'PU1': True}]   
5   [{'CEBPa': True, 'SCL': False, 'PU1': True, 'G...   
6       [{'EKLF': True, 'GATA1': True, 'PU1': False}]   
7       [{'PU1': False, 'Fli1': True, 'GATA1': True}]   
8    [{'CEBPa': True, 'EgrNab': False, 'Gfi1': True}]   
9   [{'CEBPa': False, 'EKLF': True, 'GATA1': True,...   
10  [{'GATA1': True, 'Fli1': True, 'PU1': False, '...   
11                                 [{'CEBPa': False}]   

                                            temporary  \
0                   [{'Fli1': True}, {'EKLF': False}]   
1                                     [{'PU1': True}]   
2   [{'CEBPa': True, 'cJun': False}, {'Gfi1': True...   
3                   [{'Fli1': False}, {'EKLF': True}]   
4                                     [{'PU1': True}]   
5   [{'cJun': False, 'CEBPa': True}, {'Gfi1': True...   
6   [{'PU1': False, 'GATA2': True, 'Fli1': False},...   
7                      [{'Fli1': True, 'PU1': False}]   
8   [{'CEBPa': True, 'cJun': False}, {'CEBPa': Tru...   
9   [{'Fli1': False, 'GATA2': True, 'PU1': False},...   
10                     [{'Fli1': True, 'PU1': False}]   
11                                 [{'CEBPa': False}]   

                                            permanent  
0                   [{'Fli1': True}, {'EKLF': False}]  
1                                     [{'PU1': True}]  
2   [{'CEBPa': True, 'cJun': False}, {'Gfi1': True...  
3                   [{'Fli1': False}, {'EKLF': True}]  
4                                     [{'PU1': True}]  
5   [{'CEBPa': True, 'cJun': False}, {'CEBPa': Tru...  
6   [{'GATA1': True, 'Fli1': False, 'PU1': False},...  
7                      [{'PU1': False, 'Fli1': True}]  
8   [{'cJun': False, 'CEBPa': True}, {'CEBPa': Tru...  
9   [{'Fli1': False, 'GATA1': True, 'PU1': False},...  
10                     [{'Fli1': True, 'PU1': False}]  
11                                 [{'CEBPa': False}]

In [35]:
for _, r in df.iterrows():
    print(r['source'], "->", r["target"], ':', r['permanent'])

Erythrocyte -> Megakaryocyte : [{'Fli1': True}, {'EKLF': False}]
Erythrocyte -> Monocyte : [{'PU1': True}]
Erythrocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'EgrNab': False}]
Megakaryocyte -> Erythrocyte : [{'Fli1': False}, {'EKLF': True}]
Megakaryocyte -> Monocyte : [{'PU1': True}]
Megakaryocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'EgrNab': False}]
Monocyte -> Erythrocyte : [{'GATA1': True, 'Fli1': False, 'PU1': False}, {'EKLF': True, 'PU1': False, 'GATA1': True}]
Monocyte -> Megakaryocyte : [{'PU1': False, 'Fli1': True}]
Monocyte -> Granulocyte : [{'cJun': False, 'CEBPa': True}, {'CEBPa': True, 'Gfi1': True}, {'EgrNab': False, 'CEBPa': True}]
Granulocyte -> Erythrocyte : [{'Fli1': False, 'GATA1': True, 'PU1': False}, {'GATA1': True, 'PU1': False, 'EKLF': True}]
Granulocyte -> Megakaryocyte : [{'Fli1': True, 'PU1': False}]
Granulocyte -> Monocyte : [{'CEBPa': False}]

From this data, we can (manually) derive a following source-target diagram, showing all minimal source-target controls.

![Source_target_controls](myeloid_source_target.png)

In [36]:
# We can easily compute the min sizes and options for the same sized control
df['min_onestep_size'] = df['onestep'].apply(lambda x: len(x[0]))
df['min_onestep_options'] = df['onestep'].apply(lambda x: len(x))
df['min_temporary_size'] = df['temporary'].apply(lambda x: len(x[0]))
df['min_temporary_options'] = df['temporary'].apply(lambda x: len(x))
df['min_permanent_size'] = df['permanent'].apply(lambda x: len(x[0]))
df['min_permanent_options'] = df['permanent'].apply(lambda x: len(x))
df.filter(regex='^(source|target|min_)')

source         target  min_onestep_size  min_onestep_options  \
0     Erythrocyte  Megakaryocyte                 2                    1   
1     Erythrocyte       Monocyte                 1                    1   
2     Erythrocyte    Granulocyte                 3                    1   
3   Megakaryocyte    Erythrocyte                 2                    1   
4   Megakaryocyte       Monocyte                 1                    1   
5   Megakaryocyte    Granulocyte                 4                    3   
6        Monocyte    Erythrocyte                 3                    1   
7        Monocyte  Megakaryocyte                 3                    1   
8        Monocyte    Granulocyte                 3                    1   
9     Granulocyte    Erythrocyte                 4                    1   
10    Granulocyte  Megakaryocyte                 4                    1   
11    Granulocyte       Monocyte                 1                    1   

    min_temporary_size  min_temporary_options  min_permanent_size  \
0                    1                      2                   1   
1                    1                      1                   1   
2                    2                      3                   2   
3                    1                      2                   1   
4                    1                      1                   1   
5                    2                      3                   2   
6                    3                      6                   3   
7                    2                      1                   2   
8                    2                      4                   2   
9                    3                      6                   3   
10                   2                      1                   2   
11                   1                      1                   1   

    min_permanent_options  
0                       2  
1                       1  
2                       3  
3                       2  
4                       1  
5                       3  
6                       2  
7                       1  
8                       3  
9                       2  
10                      1  
11                      1

In [37]:
# In a similar manner, we can compute phenotype control, even though this time, only target attractor
# (or phenotype, but in this case we consider attractor to be phenotype) is known.

df['phenotype'] = df['target'].apply(lambda t: pick_result(ba.Control.phenotype_permanent(
    graph=pstg,
    phenotype=all_atts[t],
    oscillation_type="forbidden",
    size_limit=10,
    stop_when_found=True
)))

In [38]:
df['min_phenotype_size'] = df['phenotype'].apply(lambda x: len(x[0]))
df['min_phenotype_options'] = df['phenotype'].apply(lambda x: len(x))

In [39]:
df.filter(regex='^(source|target|min_)')

source         target  min_onestep_size  min_onestep_options  \
0     Erythrocyte  Megakaryocyte                 2                    1   
1     Erythrocyte       Monocyte                 1                    1   
2     Erythrocyte    Granulocyte                 3                    1   
3   Megakaryocyte    Erythrocyte                 2                    1   
4   Megakaryocyte       Monocyte                 1                    1   
5   Megakaryocyte    Granulocyte                 4                    3   
6        Monocyte    Erythrocyte                 3                    1   
7        Monocyte  Megakaryocyte                 3                    1   
8        Monocyte    Granulocyte                 3                    1   
9     Granulocyte    Erythrocyte                 4                    1   
10    Granulocyte  Megakaryocyte                 4                    1   
11    Granulocyte       Monocyte                 1                    1   

    min_temporary_size  min_temporary_options  min_permanent_size  \
0                    1                      2                   1   
1                    1                      1                   1   
2                    2                      3                   2   
3                    1                      2                   1   
4                    1                      1                   1   
5                    2                      3                   2   
6                    3                      6                   3   
7                    2                      1                   2   
8                    2                      4                   2   
9                    3                      6                   3   
10                   2                      1                   2   
11                   1                      1                   1   

    min_permanent_options  min_phenotype_size  min_phenotype_options  
0                       2                   2                      1  
1                       1                   2                      1  
2                       3                   2                      3  
3                       2                   3                      2  
4                       1                   2                      1  
5                       3                   2                      3  
6                       2                   3                      2  
7                       1                   2                      1  
8                       3                   2                      3  
9                       2                   3                      2  
10                      1                   2                      1  
11                      1                   2                      1

We can use the phenotype control results to compare whether they are in align with phenotype branching observed in the source of the model

In [40]:
for a in all_atts.keys():
    pd_select = df.loc[df['target'] == a, 'phenotype'].iloc[0]
    print(a, pd_select)

Erythrocyte [{'PU1': False, 'GATA1': True, 'Fli1': False}, {'EKLF': True, 'GATA1': True, 'PU1': False}]
Megakaryocyte [{'Fli1': True, 'PU1': False}]
Monocyte [{'CEBPa': False, 'PU1': True}]
Granulocyte [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'EgrNab': False, 'CEBPa': True}]


We can also compare the results with the original study

![image](myeloid_phenotype.png)
![image](phenotypes.png)


We can see that the results are consistent, because:

- Phenotype control is very similar to the original study. Since study does not assume all initial states, the control is slightly different and there are some extra variables to control (i.e. min phenotype controls with size greater than 2) but they are not in contradiction with the results. 
- There are some extra possible ways for the phenotype control, esp. negation of the last branch
- Phenotype control is for most of the cases is a super set of the source-targets control. No source-target control is in a contradiction with the phenotype control.
    - This shows that if we know the source and specific target in advance, we can further minimize the size of the control

## Control of partially unknown model

Now let's imagine, that some functions of the original model are unknown

![image](unknowns.png)

In [41]:
# Load the model and create a Perturbation Graph

model_unknown = ba.BooleanNetwork.from_file('myeloid_3unknown.aeon')
pstg_unknown = ba.AsynchronousPerturbationGraph(model_unknown)

In [42]:
# We can easily assign the robustness to the perturbations of source-target controls

for _, r in df.iterrows():
    control = ba.Control.attractor_permanent(pstg_unknown, all_atts[r['source']], all_atts[r['target']])
    robustness = [control.perturbation_robustness(p) for p in r['permanent']]
    print(r['source'], "->", r["target"], ':', r['permanent'], ':', robustness)

Erythrocyte -> Megakaryocyte : [{'Fli1': True}, {'EKLF': False}] : [0.848605, 0.848605]
Erythrocyte -> Monocyte : [{'PU1': True}] : [0.019736]
Erythrocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'CEBPa': True, 'EgrNab': False}] : [0.122306, 0.122306, 0.122306]
Megakaryocyte -> Erythrocyte : [{'Fli1': False}, {'EKLF': True}] : [0.848605, 0.424436]
Megakaryocyte -> Monocyte : [{'PU1': True}] : [0.019736]
Megakaryocyte -> Granulocyte : [{'CEBPa': True, 'cJun': False}, {'CEBPa': True, 'Gfi1': True}, {'CEBPa': True, 'EgrNab': False}] : [0.122306, 0.122306, 0.122306]
Monocyte -> Erythrocyte : [{'GATA1': True, 'Fli1': False, 'PU1': False}, {'EKLF': True, 'PU1': False, 'GATA1': True}] : [0.642197, 0.321098]
Monocyte -> Megakaryocyte : [{'PU1': False, 'Fli1': True}] : [0.476685]
Monocyte -> Granulocyte : [{'cJun': False, 'CEBPa': True}, {'CEBPa': True, 'Gfi1': True}, {'EgrNab': False, 'CEBPa': True}] : [0.251692, 0.251692, 0.251692]
Granulocyte -> Eryth

As for permanent phenotype control, the case might be a little more complicated. Since phenotype offers a possibility to specify any subset of states. Therefore, our phenotypes (monocyte, granulocyte, megakaryocyte, erythrocyte) might correspond to more than a single state (which was the case of the original model and only possible specification for the source-target control). Let's have a look at attractors present in the phenotypes. 

In [43]:
erythrocyte = pstg_unknown.mk_subspace({ "EKLF": True })
ery_atts = ba.Attractors.attractors(pstg_unknown, erythrocyte)
ery_union = pstg_unknown.mk_empty_vertices()
for a in ery_atts:
    ery_union = ery_union.union(a.vertices())
print('erythrocyte states_count', ery_union.cardinality())

megakaryocyte = pstg_unknown.mk_subspace({ "Fli1": True })
meg_atts = ba.Attractors.attractors(pstg_unknown, megakaryocyte)
meg_union = pstg_unknown.mk_empty_vertices()
for a in meg_atts:
    meg_union = meg_union.union(a.vertices())
print('megakaryocyte states_count', meg_union.cardinality())

monocyte = pstg_unknown.mk_subspace({ "cJun": True })
mon_atts = ba.Attractors.attractors(pstg_unknown, monocyte)
mon_union = pstg_unknown.mk_empty_vertices()
for a in mon_atts:
    mon_union = mon_union.union(a.vertices())
print('monocyte states_count', mon_union.cardinality())

granulocyte = pstg_unknown.mk_subspace({ "Gfi1": True })
gra_atts = ba.Attractors.attractors(pstg_unknown, granulocyte)
gra_union = pstg_unknown.mk_empty_vertices()
for a in gra_atts:
    gra_union = gra_union.union(a.vertices())
print('granulocyte states_count', gra_union.cardinality())

erythrocyte states_count 4
megakaryocyte states_count 122
monocyte states_count 8
granulocyte states_count 73


In [44]:
# Let's verify, that the attractor states do not have any intersections
all_phenotype_atts = [ery_union, meg_union, mon_union, gra_union]
for i in range(4):
    other_atts_union = pstg_unknown.mk_empty_vertices()
    for j in range(4):
        if i == j:
            continue
        other_atts_union = other_atts_union.union(all_phenotype_atts[j])
    print('phenotype', i, 'intersect with others:', all_phenotype_atts[i].intersect(other_atts_union).cardinality())

phenotype 0 intersect with others: 2
phenotype 1 intersect with others: 28
phenotype 2 intersect with others: 4
phenotype 3 intersect with others: 26


Since the attractors within phenotypes are not disjoint (which is against our understanding of the emodel, where a call can specialize into a single cell type), we need to better specify the phenotypes for control

In [50]:
erythrocyte = pstg_unknown.mk_subspace({ "EKLF": True, "Gfi1": False, "cJun": False, "Fli1": False })
ery_atts = ba.Attractors.attractors(pstg_unknown, erythrocyte)
ery_union = pstg_unknown.mk_empty_vertices()
for a in ery_atts:
    ery_union = ery_union.union(a.vertices())
print('erythrocyte states_count', ery_union.cardinality())
print('erythrocyte states')
print('erythrocyte variables differing from the original:')
for c, a in enumerate(ery_union.items()):
    print('State #', c)
    for i, j in zip(sorted(next(erythrocyte_att.items()).to_named_dict().items()), sorted(a.to_named_dict().items())):
        if i[1] != j[1]:
            print(j[0], j[1])

megakaryocyte = pstg_unknown.mk_subspace({ "Fli1": True, "Gfi1": False, "cJun": False, "EKLF": False })
meg_atts = ba.Attractors.attractors(pstg_unknown, megakaryocyte)
meg_union = pstg_unknown.mk_empty_vertices()
for a in meg_atts:
    meg_union = meg_union.union(a.vertices())
print('megakaryocyte states_count', meg_union.cardinality())
print('megakaryocyte variables differing from the original:')
for c, a in enumerate(meg_union.items()):
    print('State #', c)
    for i, j in zip(sorted(next(megakaryocyte_att.items()).to_named_dict().items()), sorted(a.to_named_dict().items())):
        if i[1] != j[1]:
            print(j[0], j[1])

monocyte = pstg_unknown.mk_subspace({ "cJun": True, "Gfi1": False, "Fli1": False, "EKLF": False })
mon_atts = ba.Attractors.attractors(pstg_unknown, monocyte)
mon_union = pstg_unknown.mk_empty_vertices()
for a in mon_atts:
    mon_union = mon_union.union(a.vertices())
print('monocyte states_count', mon_union.cardinality())
print('monocyte variables differing from the original:')
for c, a in enumerate(mon_union.items()):
    print('State #', c)
    for i, j in zip(sorted(next(monocyte_att.items()).to_named_dict().items()), sorted(a.to_named_dict().items())):
        if i[1] != j[1]:
            print(j[0], j[1])

granulocyte = pstg_unknown.mk_subspace({ "Gfi1": True, "cJun": False, "Fli1": False, "EKLF": False })
gra_atts = ba.Attractors.attractors(pstg_unknown, granulocyte)
gra_union = pstg_unknown.mk_empty_vertices()
for a in gra_atts:
    gra_union = gra_union.union(a.vertices())
print('granulocyte states_count', gra_union.cardinality())
print('granulocyte variables differing from the original:')
for c, a in enumerate(gra_union.items()):
    print('State #', c)    
    for i, j in zip(sorted(next(granulocyte_att.items()).to_named_dict().items()), sorted(a.to_named_dict().items())):
        if i[1] != j[1]:
            print(j[0], j[1])

erythrocyte states_count 2
erythrocyte states
erythrocyte variables differing from the original:
State # 0
State # 1
GATA2 True
megakaryocyte states_count 2
megakaryocyte variables differing from the original:
State # 0
State # 1
GATA2 True
monocyte states_count 4
monocyte variables differing from the original:
State # 0
State # 1
CEBPa True
State # 2
GATA2 True
State # 3
CEBPa True
GATA2 True
granulocyte states_count 3
granulocyte variables differing from the original:
State # 0
PU1 False
State # 1
State # 2
GATA2 True


In the original article, there is also an analysis of the progenitors.

<img alt="image" src="measurements.png" width="1500"/>


From the looks, we can observe, that all states differing from the original attractor differ in variables which are exposed by progenitors. The exception is monocyte, where actually CEBPa vlaue in the original model do not correspond to the progenitor state as well as the GATA2 value which is True in for some model colors, while there is only a small hint of GATA 2 in the observed progenitor.

These small discrepancies are out of scope for this case study, but might be interesting for further analysis.

The conclusion is, that refined - more specific phenotypes are a reasonable choice for phenotype control.

We can now assign robustness to the original phenotype control

In [56]:
phen_spaces = {
    "Erythrocyte": ery_union,
    "Megakaryocyte": meg_union,
    "Monocyte": mon_union,
    "Granulocyte": gra_union
}

for a in phen_spaces.keys():
    old_perturbations = df.loc[df['target'] == a, 'phenotype'].iloc[0]
    control = ba.Control.phenotype_permanent(pstg_unknown, phen_spaces[a])
    robustness = [control.perturbation_robustness(p) for p in old_perturbations]
    print(a, old_perturbations, robustness)

Erythrocyte [{'PU1': False, 'GATA1': True, 'Fli1': False}, {'EKLF': True, 'GATA1': True, 'PU1': False}] [0.921052, 0.460526]
Megakaryocyte [{'Fli1': True, 'PU1': False}] [0.921052]
Monocyte [{'CEBPa': False, 'PU1': True}] [0.5]
Granulocyte [{'CEBPa': True, 'cJun': False}, {'Gfi1': True, 'CEBPa': True}, {'EgrNab': False, 'CEBPa': True}] [0.143044, 0.143044, 0.143044]


Now we can assign robustness to all former perturbations.

![Perturbations_robustness](perturbations_robustness.png)

We can notice, that the robustness is highly variable and in some cases perturbations of the same size significantly stand out compared to other perturbations (e.g. Megacaryocyte -> Erythrocyte)

Also, contra-intuitively, phenotype control has higher robustness than in some specific source-target perturbations. This is probably thanks to considering similar attrators as the same phenotype, what is not easily achievable in the source-target control. Here, we would need to compute source-target control for each of the attractors separately and than union the results.